In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from feature_learning import fit_svm, do_pca, fit_log_regression, cross_validate_convNN
from load_and_preprocessing import load_data_and_filter_members

In [3]:
leg='50'
from_date = None
to_date = None
link_cutoff = 0.5
data_transf, adjacency, node_index, affairs_features, sum_na_per_row = load_data_and_filter_members('../data/abdb-de-all-affairs-'+leg+'-0.csv',
                                                                     start_date=from_date, end_date=to_date,
                                                                     filter_method='number_NA',cutoff=10,ret_transf=True, delete_links_below=link_cutoff )

(Nbr. of councillors, nbr. of votes) before filter: (221, 1320)
(Nbr. of councillors, nbr. of votes) after filter: (175, 1320)


Put train data import here after

In [ ]:
labels_F = abs(GFT(X_train[167]))

plt.figure(figsize=(12, 4))

plt.bar(e, X_train[45], width=0.002)
plt.scatter(e,  X_train[45],s=20,color='red')
plt.xlabel('Eigenvalues $\lambda$', fontsize=16)
plt.ylabel('$\hat{f} (\lambda)$', fontsize=18)
plt.title('Graph Fourier Transfor of political orientation signal')
plt.axis([-0.01, 1.2, 0, 13])

In [6]:
graph = nx.from_numpy_array(adjacency)
laplacian = nx.normalized_laplacian_matrix(graph)
vals, vecs = np.linalg.eigh(laplacian.toarray())
e = vals
U = vecs

def GFT(x):
    s=U.T@x
    return s

def iGFT(x):
    s_i=U@x
    return s_i

def heat_kernel(e, t):
    kernel=np.exp(-t*e)
    return kernel

def inverse_kernel(e, t):
    kernel=1/(1+t*e)
    return kernel

def rectangle_kernel(e, l_min, l_max):
    kernel=np.zeros(e.size)
    kernel[(e>l_min)&(e<l_max)]=1
    return kernel

def graph_filter(x, kernel,e, **kwargs):
    x_hat=GFT(x)
    filtred=kernel(e,**kwargs)*x_hat
    x_filtered=iGFT(filtred)
    return x_filtered

In [16]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')

X_train = X_train[y_train != 2]
y_train = y_train[y_train !=2]

X_train = GFT(X_train.T)

X_train = X_train.T

X_train.shape

(526, 175)

In [17]:
classes, counts = np.unique(y_train, return_counts=True)
print("Classes {0}, samples per class {1}, {2}"
      .format(len(classes), counts[0], counts[1]))

Classes 2, samples per class 313, 213


**Do PCA**

In [21]:
#X = do_pca(X_train, n_comps=10, plot_expl_var=True)
X = X_train
X.shape

(526, 175)

**Do logistic regression**

In [22]:
log_reg_accuracy = fit_log_regression(X,y_train,20,k_fold=5)

Accuracy: 0.76


**Do SVM**

In [14]:
c_hyperparam = np.logspace(-6,-1,11)
use_kernel = 'linear'
k_fold = 5

mean_accuracy, variance = fit_svm(X, y_train, use_kernel, c_hyperparam, k_fold=k_fold)

Accuracy: 0.60 (+/- 0.00)
Accuracy: 0.60 (+/- 0.00)
Accuracy: 0.60 (+/- 0.00)
Accuracy: 0.60 (+/- 0.00)
Accuracy: 0.60 (+/- 0.00)
Accuracy: 0.60 (+/- 0.00)
Accuracy: 0.60 (+/- 0.00)
Accuracy: 0.60 (+/- 0.00)
Accuracy: 0.60 (+/- 0.00)
Accuracy: 0.59 (+/- 0.01)
Accuracy: 0.58 (+/- 0.03)


In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.plot(mean_accuracy, 'b.-')
ax1.set_ylabel('Accuracy', color='b')
ax2 = ax1.twinx()
ax2.plot(variance, 'g.-')
ax2.set_ylabel('training loss', color='g')
plt.show()

ConvNN:

In [ ]:
leg='50'
from_date = None
to_date = None
link_cutoff = 0.7
data_transf, adjacency, node_index, affairs_features, sum_na_per_row = load_data_and_filter_members('../data/abdb-de-all-affairs-'+leg+'-0.csv',
                                                                     start_date=from_date, end_date=to_date,
                                                                     filter_method='number_NA',cutoff=10,ret_transf=True, delete_links_below=link_cutoff )

In [ ]:
hyperparam_values = np.logspace(-7,-2,15)
hyperparam_name = 'regularization'
k_fold = 5
nn_accuracy, nn_loss = cross_validate_convNN(X_train, y_train, adjacency, hyperparam_name, hyperparam_values, k_fold)

In [ ]:
nn_accuracy

In [ ]:
fig, ax1 = plt.subplots(figsize=(15, 5))
ax1.plot(hyperparam_values, np.array(nn_accuracy)[:,1], 'b.-')
ax1.set_ylabel('Accuracy', color='b')
ax1.set_xlabel(hyperparam_name)
ax2 = ax1.twinx()
ax2.plot(hyperparam_values, np.array(nn_loss)[:,1], 'g.-')
ax2.set_ylabel('training loss', color='g')
plt.show()

In [ ]:
hyperparam_values